In [19]:
from pyspark.sql import SparkSession
import re

# Initialize SparkSession
spark = SparkSession.builder.getOrCreate()

# Sample DataFrame
data = [("I love running and playing football! #sports",)]
df = spark.createDataFrame(data, ["text"])

In [20]:
# Stemming (using Porter's algorithm)
def stem(word):
    suffixes = ['s', 'es', 'ed', 'ing', 'ly']
    for suffix in suffixes:
        if word.endswith(suffix):
            return word[:-len(suffix)]
    return word

# Lemmatization
def lemmatize(word):
    # Add your own lemmatization rules here
    lemmatized_word = word  # Placeholder, implement your logic
    return lemmatized_word

# Regex removal
def remove_regex(text):
    pattern = r"[^\w\s]"
    return re.sub(pattern, "", text)

# Hashtag removal
def remove_hashtags(text):
    pattern = r"#\w+"
    return re.sub(pattern, "", text)

# Stopwords removal
def remove_stopwords(text):
    # Hard-coded stopwords
    stopwords = ['the', 'and', 'a', 'is', 'in']  # Add your own list of stopwords
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stopwords]
    return ' '.join(filtered_words)

In [21]:
# UDFs (User-Defined Functions)
stem_udf = spark.udf.register("stem", stem)
lemmatize_udf = spark.udf.register("lemmatize", lemmatize)
remove_regex_udf = spark.udf.register("remove_regex", remove_regex)
remove_hashtags_udf = spark.udf.register("remove_hashtags", remove_hashtags)
remove_stopwords_udf = spark.udf.register("remove_stopwords", remove_stopwords)


In [22]:
# Apply transformations
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType
import re
df = df.withColumn("stemmed_text", stem_udf("text"))
df = df.withColumn("lemmatized_text", lemmatize_udf("text"))
df = df.withColumn("regex_removed_text", remove_regex_udf("text"))
df = df.withColumn("hashtags_removed_text", remove_hashtags_udf("text"))
df.show(truncate=False)

+--------------------------------------------+-------------------------------------------+--------------------------------------------+------------------------------------------+-------------------------------------+
|text                                        |stemmed_text                               |lemmatized_text                             |regex_removed_text                        |hashtags_removed_text                |
+--------------------------------------------+-------------------------------------------+--------------------------------------------+------------------------------------------+-------------------------------------+
|I love running and playing football! #sports|I love running and playing football! #sport|I love running and playing football! #sports|I love running and playing football sports|I love running and playing football! |
+--------------------------------------------+-------------------------------------------+------------------------------------------

In [25]:
from pyspark.sql import SparkSession
import pandas as pd
# Create a SparkSession
spark = SparkSession.builder.getOrCreate()

# Read CSV file into DataFrame
pandas_df = pd.read_csv('SuicideData.csv')
spark_df = spark.createDataFrame(pandas_df)

# Show the DataFrame
spark_df.show()

+--------------------+-----------+----------+
|                text|      label|      Date|
+--------------------+-----------+----------+
|Ex Wife Threateni...|    suicide|2022-05-10|
|Am I weird I don'...|non-suicide|2022-05-10|
|Finally 2020 is a...|non-suicide|2022-05-10|
|i need helpjust h...|    suicide|2022-05-10|
|Iâm so lostHell...|    suicide|2022-05-10|
|Honetly idkI dont...|    suicide|2022-05-10|
|[Trigger warning]...|    suicide|2022-05-10|
|It ends tonight.I...|    suicide|2022-05-10|
|Everyone wants to...|non-suicide|2022-05-10|
|My life is over a...|    suicide|2022-05-10|
|I took the rest o...|    suicide|2022-05-10|
|Can you imagine g...|    suicide|2022-05-10|
|Do you think gett...|    suicide|2022-05-10|
|death, continuedI...|    suicide|2022-05-11|
|Been arrested - f...|    suicide|2022-05-11|
|Fuck the verizon ...|non-suicide|2022-05-11|
|Iâm scared.   E...|    suicide|2022-05-11|
|Well, Im screwed....|non-suicide|2022-05-11|
|I'm fucked assign...|non-suicide|

In [ ]:
# Apply transformations
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType
import re
df = df.withColumn("stemmed_text", stem_udf("text"))
df = df.withColumn("lemmatized_text", lemmatize_udf("text"))
df = df.withColumn("regex_removed_text", remove_regex_udf("text"))
df = df.withColumn("hashtags_removed_text", remove_hashtags_udf("text"))
df.show(truncate=False)

# Save MongoDB

In [ ]:
spark = SparkSession.builder \
    .appName("MongoDBIntegration") \
    .config("spark.mongodb.input.uri", "mongodb://localhost:27017/Twitter.SuicideData") \
    .config("spark.mongodb.output.uri", "mongodb://localhost:27017/Twitter.SuicideData") \
    .getOrCreate()
df.write.format("mongo").mode("append").save()
